<a href="https://colab.research.google.com/github/apchavezr/16.-Aprendizaje-Profundo-para-Ciencia-de-Datos/blob/main/CNN_Tiny_ImageNet_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de una CNN con Tiny ImageNet usando PyTorch
Este notebook tiene como propósito entrenar una red neuronal convolucional (CNN) utilizando el conjunto de datos Tiny ImageNet, una versión reducida del famoso ImageNet. Esta práctica permite comprender el flujo de trabajo típico en deep learning, desde la carga de datos hasta la interpretación de resultados.

## 🎯 Objetivo
- Entrenar un modelo CNN para clasificar imágenes de 200 clases en el conjunto de datos Tiny ImageNet.
- Visualizar el desempeño del modelo mediante curvas de pérdida y precisión.
- Identificar errores comunes en la clasificación y reflexionar sobre el impacto de la arquitectura y los datos.

## 🔧 Paso 1: Preparar el entorno
Instalación de las librerías necesarias para trabajar con PyTorch y torchvision.

In [ ]:
!pip install torch torchvision

## 📥 Paso 2: Descargar y descomprimir Tiny ImageNet
Se descarga y extrae el dataset que contiene 100.000 imágenes distribuidas en 200 clases.

In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

## 📊 Paso 3: Preprocesar y dividir el conjunto de datos
Aplicamos transformaciones a las imágenes (redimensionamiento, conversión a tensores) y creamos los `DataLoader` para entrenamiento y validación.

In [ ]:

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = 'tiny-imagenet-200'
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)


## 🧠 Paso 4: Definir una arquitectura basada en ResNet
Usamos una red ResNet-18 adaptada para 200 clases, propia del conjunto Tiny ImageNet.

In [ ]:

import torch
import torch.nn as nn
import torchvision.models as models

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 200)


## ⚙️ Paso 5: Entrenamiento del modelo
Entrenamos el modelo durante 5 épocas usando `Adam` como optimizador y `CrossEntropyLoss` como función de pérdida.

In [ ]:

import torch.optim as optim
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Época {epoch+1}, pérdida: {running_loss / len(train_loader)}")


## 🔍 Paso 6: Evaluar e interpretar errores del modelo
Identificamos algunas predicciones incorrectas para entender los desafíos del modelo y los datos.

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

model.eval()
wrong_images = []
wrong_preds = []
correct_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        for i in range(len(labels)):
            if preds[i] != labels[i]:
                wrong_images.append(images[i].cpu())
                wrong_preds.append(preds[i].cpu())
                correct_labels.append(labels[i].cpu())
        if len(wrong_images) >= 5:
            break

for i in range(5):
    plt.imshow(np.transpose(wrong_images[i].numpy(), (1, 2, 0)))
    plt.title(f"Predicción: {wrong_preds[i]}, Real: {correct_labels[i]}")
    plt.axis('off')
    plt.show()


## ✅ Conclusiones
- El uso de CNN permite aprender representaciones directamente desde los datos, eliminando la necesidad de extraer características manualmente.
- La calidad de los datos y el tamaño del conjunto son determinantes en el rendimiento del modelo.
- Este tipo de ejercicios prácticos refuerzan la comprensión de los elementos críticos en el entrenamiento de modelos profundos: arquitectura, datos, métricas y errores.